In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [12]:
import numpy as np
import cv2
import pandas as pd
import os

directory_path = "/mnt/nas27/Dataset/Samsung_DM/"
origin_path = "./train_source.csv"
target_path = "./train_target.csv"
origin_csv = pd.read_csv(os.path.join(directory_path, origin_path))
target_csv = pd.read_csv(os.path.join(directory_path, target_path))

# 체스보드의 가로, 세로 내부 코너 개수
chessboard_size = (9, 6)

# 체스보드 코너 검출을 위한 변수 설정
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# 3D 체스보드 좌표 생성
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)

# 이미지와 체스보드 코너를 저장할 리스트
images = []
corners = []

num_images = len(target_csv)  # 이미지 개수를 사용

# 체스보드 이미지들을 불러와서 처리
for idx in range(50):
    img_path = target_csv.iloc[idx, 1]
    img_path = os.path.join(directory_path, img_path[2:])

    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 체스보드 코너 검출
    ret, corner = cv2.findChessboardCorners(gray, chessboard_size, None)
    
    if ret:
        corners.append(corner)
        images.append(gray)
        
        # 코너들을 더 정확하게 찾음
        corner2 = cv2.cornerSubPix(gray, corner, (11, 11), (-1, -1), criteria)
        
# 카메라 캘리브레이션
if len(corners) > 0:
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera([objp]*len(corners), corners, images[0].shape[::-1], None, None)

    # 내부 파라미터 (카메라 매트릭스) 출력
    print("Camera matrix (Intrinsic parameters):")
    print(mtx)

    # 왜곡 파라미터 출력
    print("\nDistortion coefficients (k1, k2, p1, p2):")
    print(dist)
else:
    print("No valid chessboard images found for calibration.")


No valid chessboard images found for calibration.


In [ ]:
import cv2
import numpy as np

# 카메라 캘리브레이션으로 얻은 내부 파라미터 및 왜곡 파라미터
mtx = np.array([[fx, 0, cx],
                [0, fy, cy],
                [0, 0, 1]], dtype=np.float64)  # 카메라 매트릭스
dist = np.array([k1, k2, p1, p2, k3], dtype=np.float64)  # 왜곡 계수

# 입력 이미지 읽기
input_image = cv2.imread("input_image.jpg")

# 이미지 크기
h, w = input_image.shape[:2]

# 카메라 보정
undistorted_image = cv2.undistort(input_image, mtx, dist)

# 보정된 이미지 출력
cv2.imshow("Undistorted Image", undistorted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()